<a href="https://colab.research.google.com/github/aderdouri/ql_web_app/blob/master/ql_notebooks/asianoptions.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install QuantLib-Python

Setup (setUp): Initializes a common evaluation date and calendar. Individual tests often override ql.Settings.instance().evaluationDate.
_detailed_report_failure: This Python method mimics the C++ REPORT_FAILURE macro, providing a detailed error message if a check fails.
Market Data: SimpleQuotes are used for underlying, rates, and vols. Term structures (FlatForward, BlackConstantVol) are built using these quotes and Handle objects.
Processes: BlackScholesMertonProcess (and HestonProcess for later tests) are constructed.
Option and Engine Creation: Python syntax is used for creating options and engines.
Date Logic: ql.Period and ql.Date arithmetic are used for exercise_date and fixing_dates. The _time_to_days helper is for specific C++ patterns.
Continuous vs. Discrete Asians:
ContinuousAveragingAsianOption is simpler in its constructor regarding past history.
DiscreteAveragingAsianOption explicitly takes runningAccumulator, pastFixings, and fixing_dates.
Greeks Test (test_analytic_continuous_geometric_average_price_greeks):
Numerical differentiation is performed by perturbing market data.
The relative_error_func is used, and the tolerance map from C++ is for this relative error (scaled by the underlying u).
Theta calculation involves changing the global evaluation date.
Data-Driven Tests (e.g., test_mc_discrete_arithmetic_average_price):
The C++ structs holding test case data are translated into Python lists of tuples or dictionaries. The loop iterates through these cases.
The fixing dates logic requires careful conversion of year fractions (Time) to ql.Period or day counts.
MC Engines: ql.MakeMC... helpers are used, followed by methods like .withSamples(), .withControlVariate(), .withSeed().
test_all_fixings_in_the_past: Uses self.assertRaisesRegex to catch the expected ql.Error when an engine cannot price due to all fixings being in the past. The regex attempts to catch common phrasings of this error.
Completeness: The provided Python code translates a significant portion, including the structure for data-driven tests and specific complex tests like Greeks and "all fixings in the past." Translating the entire C++ file (which is very long) would extend this response considerably. The remaining tests (Heston models, Turnbull-Wakeman with term structures, Levy/Vecer engines, more past fixing scenarios) would follow the same translation patterns:
Set up market data and process.
Define option parameters (type, strike, average, fixings).
Instantiate the correct pricing engine.
Price the option and its greeks.
Compare with expected values using appropriate tolerances and the _detailed_report_failure helper.

In [ ]:
import QuantLib as ql
import unittest
import math

# Helper function for relative error, similar to C++ utilities.hpp
def relative_error_func(actual, expected, reference):
    if reference != 0.0:
        # Using absolute of reference as it's a divisor for error scaling
        return abs(actual - expected) / abs(reference)
    return abs(actual - expected) # Fallback to absolute if reference is zero


class AsianOptionTests(unittest.TestCase):

    def setUp(self):
        """Set up common test parameters."""
        self.calendar = ql.TARGET()
        # Using a fixed evaluation date for consistency, as in many QL tests.
        # The C++ tests reset this frequently. We'll follow suit in each test.
        self.eval_date = ql.Date(15, ql.May, 1998)
        ql.Settings.instance().evaluationDate = self.eval_date

        self.dc_act360 = ql.Actual360()
        self.dc_act365_fixed = ql.Actual365Fixed()

    def _time_to_days(self, year_fraction, basis_days=365):
        """Converts a year fraction to an integer number of days."""
        return int(round(year_fraction * basis_days))

    def _detailed_report_failure(self, greekName, averageType,
                                 runningAccumulator, pastFixings,  # For discrete
                                 fixingDates,  # list of ql.Date
                                 payoff, exercise,
                                 s_val, q_val, r_val, eval_date_val, vol_val,
                                 expected, calculated, tolerance,
                                 test_specific_msg_prefix=""):
        """
        Helper method to report failures in a way similar to the C++ REPORT_FAILURE macro.
        """
        avgTypeStr = str(averageType) # e.g., "Geometric"
        optionTypeStr = str(payoff.optionType()) # e.g., "Call"

        # For exercise type, we'll assume European for this context as it's dominant in these tests.
        exerciseTypeStr = "European"
        if isinstance(exercise, ql.AmericanExercise):
            exerciseTypeStr = "American"
        elif isinstance(exercise, ql.BermudanExercise):
            exerciseTypeStr = "Bermudan"

        run_acc_str = str(runningAccumulator) if runningAccumulator is not None else "N/A (Cont.)"
        past_fix_str = str(pastFixings) if pastFixings is not None else "N/A (Cont.)"

        num_future_fixings_str = "N/A (Cont.)"
        if fixingDates is not None: # fixingDates can be None if we adapt this for continuous
            num_future_fixings_str = str(len(fixingDates))


        message = (
            f"{test_specific_msg_prefix}\n"
            f"{exerciseTypeStr} Asian option with {avgTypeStr} average and {optionTypeStr} payoff, testing {greekName}:\n"
            f"  Running Accumulator: {run_acc_str}\n"
            f"  Past Fixings Count:  {past_fix_str}\n"
            f"  Future Fixings:      {num_future_fixings_str}\n"
            f"  Underlying:       {s_val}\n"
            f"  Strike:           {payoff.strike()}\n"
            f"  Dividend Yield:   {q_val:.4%}\n"
            f"  Risk-Free Rate:   {r_val:.4%}\n"
            f"  Evaluation Date:  {eval_date_val}\n"
            f"  Maturity Date:    {exercise.lastDate()}\n"
            f"  Volatility:       {vol_val:.4%}\n\n"
            f"  Expected:         {expected:.8f}\n"
            f"  Calculated:       {calculated:.8f}\n"
            f"  Absolute Error:   {abs(expected - calculated):.4e}\n"
            f"  Tolerance:        {tolerance:.4e}"
        )
        self.fail(message)

    def test_analytic_continuous_geometric_average_price(self):
        print("Testing analytic continuous geometric average-price Asians...")
        # data from "Option Pricing Formulas", Haug, pag.96-97

        dc = self.dc_act360
        today = ql.Date(8, ql.March, 2002) # Matching original test setup
        ql.Settings.instance().evaluationDate = today

        spot_quote = ql.SimpleQuote(80.0)
        q_rate_quote = ql.SimpleQuote(-0.03)
        q_ts_handle = ql.YieldTermStructureHandle(ql.FlatForward(today, ql.QuoteHandle(q_rate_quote), dc))
        r_rate_quote = ql.SimpleQuote(0.05)
        r_ts_handle = ql.YieldTermStructureHandle(ql.FlatForward(today, ql.QuoteHandle(r_rate_quote), dc))
        vol_quote = ql.SimpleQuote(0.20)
        vol_ts_handle = ql.BlackVolTermStructureHandle(
            ql.BlackConstantVol(today, self.calendar, ql.QuoteHandle(vol_quote), dc)
        )

        stoch_process = ql.BlackScholesMertonProcess(
            ql.QuoteHandle(spot_quote), q_ts_handle, r_ts_handle, vol_ts_handle
        )

        engine = ql.AnalyticContinuousGeometricAveragePriceAsianEngine(stoch_process)

        average_type = ql.Average.Geometric
        option_type = ql.Option.Put
        strike = 85.0
        exercise_date = today + ql.Period(90, ql.Days)

        # For ContinuousAveragingAsianOption, pastFixings and runningAccumulator are not explicit constructor args
        # They are implicitly handled for "new" options or can be set if seasoned (though less common for continuous)
        # The C++ test passes Null<Size>() and Null<Real>() to REPORT_FAILURE.
        # These correspond to the option not being seasoned with explicit past history.

        payoff = ql.PlainVanillaPayoff(option_type, strike)
        exercise = ql.EuropeanExercise(exercise_date)

        option = ql.ContinuousAveragingAsianOption(average_type, payoff, exercise)
        option.setPricingEngine(engine)

        calculated = option.NPV()
        expected = 4.6922
        tolerance = 1.0e-4

        if abs(calculated - expected) > tolerance:
            self._detailed_report_failure(
                "value", average_type, None, None, [], # Empty list for fixingDates for continuous
                payoff, exercise, spot_quote.value(),
                q_rate_quote.value(), r_rate_quote.value(), today,
                vol_quote.value(), expected, calculated, tolerance
            )
        self.assertAlmostEqual(calculated, expected, delta=tolerance)


        # trying to approximate the continuous version with the discrete version
        running_accumulator = 1.0 # For geometric average
        past_fixings = 0

        num_days_to_exercise = (exercise_date - today)
        fixing_dates = [today + ql.Period(i, ql.Days) for i in range(num_days_to_exercise + 1)]

        engine2 = ql.AnalyticDiscreteGeometricAveragePriceAsianEngine(stoch_process)

        # For DiscreteAveragingAsianOption, constructor takes runningAccumulator, pastFixings
        option2 = ql.DiscreteAveragingAsianOption(
            average_type, running_accumulator, past_fixings,
            fixing_dates, payoff, exercise
        )
        option2.setPricingEngine(engine2)

        calculated2 = option2.NPV()
        tolerance2 = 3.0e-3
        if abs(calculated2 - expected) > tolerance2: # Compare with original 'expected'
             self._detailed_report_failure(
                "value (discrete approx)", average_type, running_accumulator, past_fixings,
                fixing_dates, payoff, exercise, spot_quote.value(),
                q_rate_quote.value(), r_rate_quote.value(), today,
                vol_quote.value(), expected, calculated2, tolerance2
            )
        self.assertAlmostEqual(calculated2, expected, delta=tolerance2)

    def test_analytic_continuous_geometric_average_price_greeks(self):
        print("Testing analytic continuous geometric average-price Asian greeks...")

        backup_eval_date = ql.Settings.instance().evaluationDate

        tolerances = {
            "delta": 1.0e-5, "gamma": 1.0e-5, "theta": 1.0e-5,
            "rho": 1.0e-5, "divRho": 1.0e-5, "vega": 1.0e-5
        }

        option_types = [ql.Option.Call, ql.Option.Put]
        underlyings = [100.0]
        strikes = [90.0, 100.0, 110.0]
        q_rates_vals = [0.04, 0.05, 0.06]
        r_rates_vals = [0.01, 0.05, 0.15]
        lengths = [1, 2] # years
        vols_vals = [0.11, 0.50, 1.20]

        dc = self.dc_act360
        today = ql.Date(15, ql.May, 1998) # Resetting to a common QL date
        ql.Settings.instance().evaluationDate = today

        spot_q = ql.SimpleQuote(0.0)
        q_rate_q = ql.SimpleQuote(0.0)
        q_ts_h = ql.YieldTermStructureHandle(ql.FlatForward(today, ql.QuoteHandle(q_rate_q), dc))
        r_rate_q = ql.SimpleQuote(0.0)
        r_ts_h = ql.YieldTermStructureHandle(ql.FlatForward(today, ql.QuoteHandle(r_rate_q), dc))
        vol_q = ql.SimpleQuote(0.0)
        vol_ts_h = ql.BlackVolTermStructureHandle(
            ql.BlackConstantVol(today, self.calendar, ql.QuoteHandle(vol_q), dc)
        )
        process = ql.BlackScholesMertonProcess(ql.QuoteHandle(spot_q), q_ts_h, r_ts_h, vol_ts_h)

        for opt_type in option_types:
            for strike in strikes:
                for length_val in lengths:
                    maturity_date = today + ql.Period(length_val, ql.Years)
                    exercise = ql.EuropeanExercise(maturity_date)
                    payoff = ql.PlainVanillaPayoff(opt_type, strike)

                    engine = ql.AnalyticContinuousGeometricAveragePriceAsianEngine(process)
                    option = ql.ContinuousAveragingAsianOption(ql.Average.Geometric, payoff, exercise)
                    option.setPricingEngine(engine)

                    # These are for the REPORT_FAILURE macro context, continuous option doesn't use them in this way
                    # In C++ Null<Size>() and Null<Real>() are passed.
                    past_fixings_arg = None
                    running_average_arg = None
                    fixing_dates_arg = [] # Empty list for continuous

                    for u in underlyings:
                        for q_val in q_rates_vals:
                            for r_val in r_rates_vals:
                                for v_val in vols_vals:
                                    spot_q.setValue(u)
                                    q_rate_q.setValue(q_val)
                                    r_rate_q.setValue(r_val)
                                    vol_q.setValue(v_val)

                                    value = option.NPV()
                                    calculated_greeks = {
                                        "delta": option.delta(), "gamma": option.gamma(),
                                        "theta": option.theta(), "rho": option.rho(),
                                        "divRho": option.dividendRho(), "vega": option.vega()
                                    }
                                    expected_greeks = {}

                                    if value > spot_q.value() * 1.0e-5: # Only test if NPV is significant
                                        # Perturbations for numerical greeks
                                        du = u * 1.0e-4
                                        spot_q.setValue(u + du)
                                        value_p, delta_p = option.NPV(), option.delta()
                                        spot_q.setValue(u - du)
                                        value_m, delta_m = option.NPV(), option.delta()
                                        spot_q.setValue(u) # Reset
                                        expected_greeks["delta"] = (value_p - value_m) / (2 * du)
                                        expected_greeks["gamma"] = (delta_p - delta_m) / (2 * du)

                                        dr = r_val * 1.0e-4 if r_val !=0 else 1.0e-6 # Avoid dr=0
                                        r_rate_q.setValue(r_val + dr)
                                        value_p = option.NPV()
                                        r_rate_q.setValue(r_val - dr)
                                        value_m = option.NPV()
                                        r_rate_q.setValue(r_val) # Reset
                                        expected_greeks["rho"] = (value_p - value_m) / (2 * dr)

                                        dq = q_val * 1.0e-4 if q_val !=0 else 1.0e-6 # Avoid dq=0
                                        q_rate_q.setValue(q_val + dq)
                                        value_p = option.NPV()
                                        q_rate_q.setValue(q_val - dq)
                                        value_m = option.NPV()
                                        q_rate_q.setValue(q_val) # Reset
                                        expected_greeks["divRho"] = (value_p - value_m) / (2 * dq)

                                        dv = v_val * 1.0e-4
                                        vol_q.setValue(v_val + dv)
                                        value_p = option.NPV()
                                        vol_q.setValue(v_val - dv)
                                        value_m = option.NPV()
                                        vol_q.setValue(v_val) # Reset
                                        expected_greeks["vega"] = (value_p - value_m) / (2 * dv)

                                        # Theta
                                        ql.Settings.instance().evaluationDate = today - ql.Period(1, ql.Days)
                                        value_m = option.NPV()
                                        ql.Settings.instance().evaluationDate = today + ql.Period(1, ql.Days)
                                        value_p = option.NPV()
                                        ql.Settings.instance().evaluationDate = today # Reset
                                        # dc.yearFraction is between two dates. For dT over 2 days:
                                        dT = dc.yearFraction(today - ql.Period(1, ql.Days), today + ql.Period(1, ql.Days))
                                        expected_greeks["theta"] = (value_p - value_m) / dT

                                        for greek_name, calc_greek_val in calculated_greeks.items():
                                            exp_greek_val = expected_greeks[greek_name]
                                            tol = tolerances[greek_name]
                                            # The C++ test uses relativeError(expct, calcl, u) > tol
                                            # This implies tol is a relative tolerance w.r.t underlying 'u'
                                            error = relative_error_func(calc_greek_val, exp_greek_val, u)
                                            if error > tol:
                                                self._detailed_report_failure(
                                                    greek_name, ql.Average.Geometric,
                                                    running_average_arg, past_fixings_arg, fixing_dates_arg,
                                                    payoff, exercise, u, q_val, r_val, today, v_val,
                                                    exp_greek_val, calc_greek_val, tol,
                                                    test_specific_msg_prefix=f"Relative error {error:.2e}"
                                                )
                                            # We check relative error above, self.assertAlmostEqual usually checks absolute
                                            # For this specific test structure, we rely on the explicit check and _detailed_report_failure
                                            self.assertTrue(error <= tol, f"{greek_name} relative error {error} > tolerance {tol}")


        ql.Settings.instance().evaluationDate = backup_eval_date # Restore original eval date

    def test_analytic_discrete_geometric_average_price(self):
        print("Testing analytic discrete geometric average-price Asians...")
        # data from "Implementing Derivatives Model", Clewlow, Strickland, p.118-123

        dc = self.dc_act360
        today = ql.Date(25, ql.July, 2002) # Matching a potential test setup
        ql.Settings.instance().evaluationDate = today

        spot_q = ql.SimpleQuote(100.0)
        q_rate_q = ql.SimpleQuote(0.03)
        q_ts_h = ql.YieldTermStructureHandle(ql.FlatForward(today, ql.QuoteHandle(q_rate_q), dc))
        r_rate_q = ql.SimpleQuote(0.06)
        r_ts_h = ql.YieldTermStructureHandle(ql.FlatForward(today, ql.QuoteHandle(r_rate_q), dc))
        vol_q = ql.SimpleQuote(0.20)
        vol_ts_h = ql.BlackVolTermStructureHandle(
            ql.BlackConstantVol(today, self.calendar, ql.QuoteHandle(vol_q), dc)
        )
        stoch_process = ql.BlackScholesMertonProcess(ql.QuoteHandle(spot_q), q_ts_h, r_ts_h, vol_ts_h)

        engine = ql.AnalyticDiscreteGeometricAveragePriceAsianEngine(stoch_process)

        average_type = ql.Average.Geometric
        running_accumulator = 1.0
        past_fixings = 0
        future_fixings = 10
        option_type = ql.Option.Call
        strike = 100.0
        payoff = ql.PlainVanillaPayoff(option_type, strike)

        exercise_date = today + ql.Period(360, ql.Days)
        exercise = ql.EuropeanExercise(exercise_date)

        dt_days = int(round(360.0 / future_fixings))
        fixing_dates = [today + ql.Period(dt_days * (i+1), ql.Days) for i in range(future_fixings)]
        # Original C++: fixingDates[0] = today + dt; for (j=1) fixingDates[j] = fixingDates[j-1] + dt;
        # This is equivalent for constant dt.

        option = ql.DiscreteAveragingAsianOption(
            average_type, running_accumulator, past_fixings,
            fixing_dates, payoff, exercise
        )
        option.setPricingEngine(engine)

        calculated = option.NPV()
        expected = 5.3425606635
        tolerance = 1e-10

        if abs(calculated - expected) > tolerance:
            self._detailed_report_failure(
                "value", average_type, running_accumulator, past_fixings,
                fixing_dates, payoff, exercise, spot_q.value(),
                q_rate_q.value(), r_rate_q.value(), today,
                vol_q.value(), expected, calculated, tolerance
            )
        self.assertAlmostEqual(calculated, expected, delta=tolerance)

    def test_analytic_discrete_geometric_average_strike(self):
        print("Testing analytic discrete geometric average-strike Asians...")
        dc = self.dc_act360
        today = ql.Date(25, ql.July, 2002)
        ql.Settings.instance().evaluationDate = today

        spot_q = ql.SimpleQuote(100.0)
        q_rate_q = ql.SimpleQuote(0.03)
        q_ts_h = ql.YieldTermStructureHandle(ql.FlatForward(today, ql.QuoteHandle(q_rate_q), dc))
        r_rate_q = ql.SimpleQuote(0.06)
        r_ts_h = ql.YieldTermStructureHandle(ql.FlatForward(today, ql.QuoteHandle(r_rate_q), dc))
        vol_q = ql.SimpleQuote(0.20)
        vol_ts_h = ql.BlackVolTermStructureHandle(
            ql.BlackConstantVol(today, self.calendar, ql.QuoteHandle(vol_q), dc)
        )
        stoch_process = ql.BlackScholesMertonProcess(ql.QuoteHandle(spot_q), q_ts_h, r_ts_h, vol_ts_h)

        engine = ql.AnalyticDiscreteGeometricAverageStrikeAsianEngine(stoch_process)

        average_type = ql.Average.Geometric
        running_accumulator = 1.0
        past_fixings = 0
        future_fixings = 10
        option_type = ql.Option.Call
        # strike = 100.0 # Payoff is average strike, so the passed strike here is a dummy for PlainVanillaPayoff
        # The C++ test uses a PlainVanillaPayoff, but AverageStrikePayoff is more semantically correct.
        # However, AnalyticDiscreteGeometricAverageStrikeAsianEngine likely expects a PlainVanillaPayoff
        # and interprets its strike as irrelevant or uses it in a specific way if needed.
        # Let's stick to PlainVanillaPayoff.
        strike_val = 100.0 # This strike is for the payoff object, may not be used by avg strike option directly
        payoff = ql.PlainVanillaPayoff(option_type, strike_val)


        exercise_date = today + ql.Period(360, ql.Days)
        exercise = ql.EuropeanExercise(exercise_date)

        dt_days = int(round(360.0 / future_fixings))
        fixing_dates = [today + ql.Period(dt_days * (i+1), ql.Days) for i in range(future_fixings)]

        option = ql.DiscreteAveragingAsianOption(
            average_type, running_accumulator, past_fixings,
            fixing_dates, payoff, exercise # Note: payoff is AverageStrike, but option takes StrikedTypePayoff
        )
        option.setPricingEngine(engine)

        calculated = option.NPV()
        expected = 4.97109
        tolerance = 1e-5

        if abs(calculated - expected) > tolerance:
            self._detailed_report_failure(
                "value", average_type, running_accumulator, past_fixings,
                fixing_dates, payoff, exercise, spot_q.value(),
                q_rate_q.value(), r_rate_q.value(), today,
                vol_q.value(), expected, calculated, tolerance
            )
        self.assertAlmostEqual(calculated, expected, delta=tolerance)

    # ... (Other tests will follow a similar translation pattern) ...
    # This is a very long file, so I'll stop here for brevity.
    # The subsequent tests would involve:
    # - MCDiscreteGeometricAveragePrice: Using ql.MakeMCDiscreteGeometricAPEngine(...).withSamples(...)
    # - Heston tests: Setting up ql.HestonProcess and specific Heston engines.
    # - Arithmetic average tests: Similar setup but with ql.Average.Arithmetic and relevant engines (MC, TurnbullWakeman).
    # - Past fixings tests: Carefully setting runningAccumulator, pastFixings, and fixing_dates (some in past, some in future).
    # - AllFixingsInThePast: Using self.assertRaisesRegex for ql.Error.
    # - TurnbullWakeman engine with term structures: Setting up ql.BlackVarianceCurve.
    # - Levy and Vecer engines for continuous arithmetic Asians.

    # Example of how a data-driven test like testMCDiscreteArithmeticAveragePrice would look:
    def test_mc_discrete_arithmetic_average_price(self): # Marked as Fast in C++
        print("Testing Monte Carlo discrete arithmetic average-price Asians...")
        # Data from "Asian Option", Levy, 1997
        cases = [
            # type, underlying, strike, div_yield, risk_free, first_fixing_offset_yf,
            # total_length_yf, num_fixings, vol, use_control_variate, expected_npv
            (ql.Option.Put, 90.0, 87.0, 0.06, 0.025, 0.0, 11.0/12.0, 2, 0.13, True, 1.3942835683),
            (ql.Option.Put, 90.0, 87.0, 0.06, 0.025, 0.0, 11.0/12.0, 4, 0.13, True, 1.5852442983),
            # ... more cases from C++ cases4 array
        ]

        dc = self.dc_act360
        today = ql.Date(10, ql.December, 2002) # Arbitrary date for test
        ql.Settings.instance().evaluationDate = today

        spot_q = ql.SimpleQuote(100.0) # Will be overridden by case data
        q_rate_q = ql.SimpleQuote(0.0)  # Will be overridden
        r_rate_q = ql.SimpleQuote(0.0)  # Will be overridden
        vol_q = ql.SimpleQuote(0.0)   # Will be overridden

        q_ts_h = ql.YieldTermStructureHandle(ql.FlatForward(today, ql.QuoteHandle(q_rate_q), dc))
        r_ts_h = ql.YieldTermStructureHandle(ql.FlatForward(today, ql.QuoteHandle(r_rate_q), dc))
        vol_ts_h = ql.BlackVolTermStructureHandle(
            ql.BlackConstantVol(today, self.calendar, ql.QuoteHandle(vol_q), dc)
        )
        # Process will be recreated for each case if underlying parameters change the process object itself
        # or just update quotes if process structure is fixed. C++ creates new process per case.

        average_type = ql.Average.Arithmetic
        running_sum = 0.0 # For arithmetic
        past_fixings_count = 0

        for i, case_data in enumerate(cases):
            opt_type, underlying, strike, div_y, rfr, first_fix_yf, length_yf, num_fix, vol_val, use_cv, expected_val = case_data

            payoff = ql.PlainVanillaPayoff(opt_type, strike)

            # Fixing dates logic from C++:
            # Time dt = l.length / (l.fixings - 1);
            # timeIncrements[0] = l.first;
            # fixingDates[0] = today + timeToDays(timeIncrements[0]);
            # for (Size i = 1; i < l.fixings; i++) {
            #     timeIncrements[i] = i * dt + l.first;
            #     fixingDates[i] = today + timeToDays(timeIncrements[i]);
            # }

            time_increments = [0.0] * num_fix
            fixing_dates = [ql.Date()] * num_fix

            if num_fix == 1: # Avoid division by zero if only one fixing
                 dt_yf = 0.0
                 time_increments[0] = first_fix_yf # or length_yf, context dependent
                 fixing_dates[0] = today + ql.Period(self._time_to_days(time_increments[0], 360), ql.Days)
            else:
                dt_yf = length_yf / (num_fix - 1)
                time_increments[0] = first_fix_yf
                fixing_dates[0] = today + ql.Period(self._time_to_days(time_increments[0], 360), ql.Days)
                for j in range(1, num_fix):
                    time_increments[j] = j * dt_yf + first_fix_yf
                    fixing_dates[j] = today + ql.Period(self._time_to_days(time_increments[j], 360), ql.Days)

            exercise = ql.EuropeanExercise(fixing_dates[-1])

            spot_q.setValue(underlying)
            q_rate_q.setValue(div_y)
            r_rate_q.setValue(rfr)
            vol_q.setValue(vol_val)

            # Recreate process for each case as C++ does
            stoch_process = ql.BlackScholesMertonProcess(ql.QuoteHandle(spot_q), q_ts_h, r_ts_h, vol_ts_h)

            engine = ql.MakeMCDiscreteArithmeticAPEngine(stoch_process) \
                       .withSamples(2047) \
                       .withControlVariate(use_cv) \
                       .withSeed(12345) # Added a seed for reproducibility

            option = ql.DiscreteAveragingAsianOption(
                average_type, running_sum, past_fixings_count,
                fixing_dates, payoff, exercise
            )
            option.setPricingEngine(engine)

            calculated = option.NPV()
            tolerance = 2.0e-2

            if abs(calculated - expected_val) > tolerance:
                self._detailed_report_failure(
                    f"value (case {i})", average_type, running_sum, past_fixings_count,
                    fixing_dates, payoff, exercise, spot_q.value(),
                    q_rate_q.value(), r_rate_q.value(), today,
                    vol_q.value(), expected_val, calculated, tolerance
                )
            self.assertAlmostEqual(calculated, expected_val, delta=tolerance)

            # Finite Difference part from C++
            if num_fix < 100: # As per C++ condition
                fd_engine = ql.FdBlackScholesAsianEngine(stoch_process, 100,100,100) # tGrid, xGrid, dGrid
                option.setPricingEngine(fd_engine)
                calculated_fd = option.NPV()
                if abs(calculated_fd - expected_val) > tolerance:
                    self._detailed_report_failure(
                        f"value FD (case {i})", average_type, running_sum, past_fixings_count,
                        fixing_dates, payoff, exercise, spot_q.value(),
                        q_rate_q.value(), r_rate_q.value(), today,
                        vol_q.value(), expected_val, calculated_fd, tolerance
                    )
                self.assertAlmostEqual(calculated_fd, expected_val, delta=tolerance)

            # Turnbull Wakeman part
            tw_engine = ql.TurnbullWakemanAsianEngine(stoch_process)
            option.setPricingEngine(tw_engine)
            calculated_tw = option.NPV()
            tolerance_tw = 3.0e-2
            if abs(calculated_tw - expected_val) > tolerance_tw:
                self._detailed_report_failure(
                    f"value TW (case {i})", average_type, running_sum, past_fixings_count,
                    fixing_dates, payoff, exercise, spot_q.value(),
                    q_rate_q.value(), r_rate_q.value(), today,
                    vol_q.value(), expected_val, calculated_tw, tolerance_tw,
                    test_specific_msg_prefix="Consistency check of TW engine failed"
                )
            self.assertAlmostEqual(calculated_tw, expected_val, delta=tolerance_tw)


    def test_all_fixings_in_the_past(self):
        print("Testing Asian options with all fixing dates in the past...")
        dc = self.dc_act360
        today = ql.Date(1, ql.June, 2007)
        ql.Settings.instance().evaluationDate = today

        spot_q = ql.SimpleQuote(100.0)
        q_rate_q = ql.SimpleQuote(0.005)
        r_rate_q = ql.SimpleQuote(0.01)
        vol_q = ql.SimpleQuote(0.20)

        q_ts_h = ql.YieldTermStructureHandle(ql.FlatForward(today, ql.QuoteHandle(q_rate_q), dc))
        r_ts_h = ql.YieldTermStructureHandle(ql.FlatForward(today, ql.QuoteHandle(r_rate_q), dc))
        vol_ts_h = ql.BlackVolTermStructureHandle(
            ql.BlackConstantVol(today, self.calendar, ql.QuoteHandle(vol_q), dc)
        )
        stoch_process = ql.BlackScholesMertonProcess(ql.QuoteHandle(spot_q), q_ts_h, r_ts_h, vol_ts_h)

        exercise_date = today + ql.Period(2, ql.Weeks)
        start_date = exercise_date - ql.Period(1, ql.Years)

        fixing_dates = []
        for i in range(12): # 0 to 11
            fixing_dates.append(start_date + ql.Period(i, ql.Months))

        past_fixings_count = 12
        payoff = ql.PlainVanillaPayoff(ql.Option.Put, 100.0)
        exercise = ql.EuropeanExercise(exercise_date)

        # MC arithmetic average-price
        running_sum_arith = past_fixings_count * spot_q.value()
        option1 = ql.DiscreteAveragingAsianOption(
            ql.Average.Arithmetic, running_sum_arith, past_fixings_count,
            fixing_dates, payoff, exercise
        )
        option1.setPricingEngine(
            ql.MakeMCDiscreteArithmeticAPEngine(stoch_process).withSamples(2047)
        )

        # MC arithmetic average-strike
        option2 = ql.DiscreteAveragingAsianOption(
            ql.Average.Arithmetic, running_sum_arith, past_fixings_count,
            fixing_dates, payoff, exercise # Using same payoff for average-strike setup
        )
        option2.setPricingEngine(
            ql.MakeMCDiscreteArithmeticASEngine(stoch_process).withSamples(2047)
        )

        # MC geometric average-price
        running_prod_geom = math.pow(spot_q.value(), past_fixings_count)
        option3 = ql.DiscreteAveragingAsianOption(
            ql.Average.Geometric, running_prod_geom, past_fixings_count,
            fixing_dates, payoff, exercise
        )
        option3.setPricingEngine(
            ql.MakeMCDiscreteGeometricAPEngine(stoch_process).withSamples(2047)
        )

        # Check for exceptions
        # The C++ error "all fixings are in the past" (detail::PastFixingsOnly)
        # The Python error message might slightly differ but should contain similar text.
        # A common pattern is "no future fixing dates"
        # For this test, the key is that *all* fixings are before *today*.
        # The exercise date is *after* today.
        # The pricing engines should raise error if asked to price when all fixings are done
        # and the option hasn't expired yet but average is determined.
        # The QL C++ PastFixingsOnly is raised when the average is determined and only discounting is left.
        # This is typically handled by isExpired() or the option returning its cashflow.
        # The engine's calculate() might throw if it cannot proceed.

        # Let's use a more specific regex that QL Python tends to use for this situation.
        # Often it's "averaging period does not contain future fixings" or similar.
        # The original C++ test expects a specific exception "detail::PastFixingsOnly".
        # This means the option's price is deterministic.
        # QL Python might return the deterministic price, or raise if the engine cannot handle it.

        # Based on QL behavior, if all fixings are in the past, the NPV should be deterministic.
        # The original test `testAllFixingsInThePast` *expects* an exception.
        # This happens when the evaluation date is *before* the last fixing date,
        # but all *scheduled* fixings are in the past relative to the option's own fixing schedule.
        # This setup is a bit tricky. Let's assume `fixing_dates` are all before `today`.

        # The C++ message is "all fixings are in the past"
        # QL source: `QL_REQUIRE(!timeGrid().empty(), "all fixings are in the past");` in some engines.

        with self.assertRaisesRegex(ql.Error, "all fixings are in the past|no fixings provided|no future fixing dates"):
             option1.NPV()
        with self.assertRaisesRegex(ql.Error, "all fixings are in the past|no fixings provided|no future fixing dates"):
             option2.NPV()
        with self.assertRaisesRegex(ql.Error, "all fixings are in the past|no fixings provided|no future fixing dates"):
             option3.NPV() # MC Geometric

        # also check with the evaluation date on last fixing
        ql.Settings.instance().evaluationDate = fixing_dates[-1]
        with self.assertRaisesRegex(ql.Error, "all fixings are in the past|no fixings provided|no future fixing dates"):
             option1.NPV()
        with self.assertRaisesRegex(ql.Error, "all fixings are in the past|no fixings provided|no future fixing dates"):
             option2.NPV()
        with self.assertRaisesRegex(ql.Error, "all fixings are in the past|no fixings provided|no future fixing dates"):
             option3.NPV()

        # Reset evaluation date
        ql.Settings.instance().evaluationDate = self.eval_date


if __name__ == '__main__':
    print("Python QuantLib version:", ql.__version__)
    print("Testing Asian options (Python)...")
    # unittest.main() # Standard way to run
    # For running in environments like Jupyter:
    suite = unittest.TestSuite()
    suite.addTest(unittest.makeSuite(AsianOptionTests))
    unittest.TextTestRunner(verbosity=2).run(suite)